# Original Dataset - ML Test

In [45]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

import sqlalchemy
from sqlalchemy import create_engine, inspect

import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import time
from datetime import datetime

import numpy as np
from joblib import dump, load
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score

from imblearn.over_sampling import SMOTE
import json

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

In [3]:
# Check for untitled_project folder
if os.path.exists("untitled_project"):
    input("DELETE untitled_project")

## Import datasets

In [4]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['alexaval',
 'alexbval',
 'alexgval',
 'alexrval',
 'aval',
 'bval',
 'chroma1',
 'chroma10',
 'chroma11',
 'chroma12',
 'chroma2',
 'chroma3',
 'chroma4',
 'chroma5',
 'chroma6',
 'chroma7',
 'chroma8',
 'chroma9',
 'chromastd',
 'deltachroma1',
 'deltachroma10',
 'deltachroma11',
 'deltachroma12',
 'deltachroma2',
 'deltachroma3',
 'deltachroma4',
 'deltachroma5',
 'deltachroma6',
 'deltachroma7',
 'deltachroma8',
 'deltachroma9',
 'deltachromastd',
 'deltaenergy',
 'deltaenergyentropy',
 'deltamfcc1',
 'deltamfcc10',
 'deltamfcc11',
 'deltamfcc12',
 'deltamfcc13',
 'deltamfcc2',
 'deltamfcc3',
 'deltamfcc4',
 'deltamfcc5',
 'deltamfcc6',
 'deltamfcc7',
 'deltamfcc8',
 'deltamfcc9',
 'deltaspectralcentroid',
 'deltaspectralentropy',
 'deltaspectralflux',
 'deltaspectralrolloff',
 'deltaspectralspread',
 'deltazcr',
 'demographic',
 'diagnosis',
 'energy',
 'energyentropy',
 'gval',
 'habits',
 'mfcc1',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5

In [5]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

In [6]:
# Merge dataframes
merged_df = pd.merge(
    dataframes['demographic_df'],
    dataframes['diagnosis_df'],
    how = 'inner',
    on = 'id'
)

merged_df = pd.merge(
    merged_df,
    dataframes['habits_df'],
    how = 'inner',
    on = 'id'
)

# Display merged_df
merged_df.head()

,id,age,gender,occupation_status,diagnosis,subtype,vhi_score,rsi_score,reflux_indicated,vhi_zscore,...,chocolate,chocolate_grams_pd,coffee,coffee_pd,citrus_fruits,citrus_fruits_pd,soft_cheese,soft_cheese_pd,tomatoes,water_litres_pd
0,voice100,24,m,unknown,healthy,no subtype,0,5,0,-0.58,...,sometimes,30,always,3,never,0.00,almost always,100,never,1.5
1,voice101,60,m,unknown,healthy,no subtype,80,10,0,4.76,...,sometimes,30,always,4,never,0.00,sometimes,100,sometimes,1.5
2,voice192,22,m,cook,hyperkinetic dysphonia,no subtype,0,10,0,-0.58,...,always,14,always,3,almost always,1.17,sometimes,100,sometimes,2.5
3,voice193,46,f,housewife,hyperkinetic dysphonia,no subtype,0,36,1,-0.58,...,sometimes,30,always,2,sometimes,1.00,sometimes,100,sometimes,1.0
4,voice008,51,f,researcher,reflux laryngitis,no subtype,19,15,1,0.68,...,almost always,20,always,2,almost always,1.00,sometimes,100,almost always,1.0


In [7]:
merged_df['diagnosis'].value_counts()

diagnosis
hyperkinetic dysphonia    70
healthy                   57
hypokinetic dysphonia     39
reflux laryngitis         38
Name: count, dtype: int64

## Preprocessing

### Separate the target and feature variables

In [8]:
# Drop the 'id' column
no_id_df = merged_df.drop(columns=['id'])

In [9]:
# Define the target variables
target_var = ['diagnosis', 'subtype']
y = no_id_df[target_var]

# Define the feature variables
X = no_id_df.drop(columns=target_var)

### Binary classification - `diagnosis`

In [10]:
# Encode the target variable, ignore subtype
y = y['diagnosis'].apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Bin `occupation_status` column

In [11]:
# Use limit_unique() function to bin the column
limit_unique(X, 10, ['occupation_status'])

occupation_status
unknown               41
researcher            41
other                 25
employee              25
housewife             23
student               16
technical operator    12
singer                10
pensioner              6
doctor                 5
Name: count, dtype: int64
Number of unique values: 10



### Encode feature columns

#### Encoding
- `smoker` column
	- `0` for `no`
	- `1` for `casual`
	- `2` for `yes`
- `alcohol_consumption` column
	- `0` for `nondrinker`
	- `1` for `casual`
	- `2` for `habitual`
- `carbonated_beverages`, `tomatoes`, `coffee`, `chocolate`, `soft_cheese`, `citrus_fruits` columns
	- `0` for `never`
	- `1` for `almost never`
	- `2` for `sometimes`
	- `3` for `almost always`
	- `4` for `always`

In [12]:
# Define the maps
smoker_map = {
    'no': 0,
    'casual': 1,
    'yes': 2
}

alcohol_map = {
    'nondrinker': 0,
    'casual': 1,
    'habitual': 2
}

habit_map = {
    'never': 0,
    'almost never': 1,
    'sometimes': 2,
    'almost always': 3,
    'always': 4
}

In [13]:
# Apply the label encoding using the maps
X['smoker'] = X['smoker'].map(smoker_map)
X['alcohol_consumption'] = X['alcohol_consumption'].map(alcohol_map)

# Habit columns
habit_cols = [
    'carbonated_beverages', 'tomatoes',
    'coffee', 'chocolate',
    'soft_cheese', 'citrus_fruits'
]

# Use a loop for the habit columns
for habit in habit_cols:
    X[habit] = X[habit].map(habit_map)

In [14]:
# Remove the occupation_status column
# X = X.drop(columns=['occupation_status'])

In [15]:
# Encode the categorical columns using get_dummies
categorical_hot = ['gender', 'occupation_status']
# categorical_hot = ['gender']

# One-hot encoding
encoded_columns = pd.get_dummies(X[categorical_hot]).astype(int)

# Update the feature dataframe
X.drop(categorical_hot, axis=1, inplace=True)
X = pd.concat([X, encoded_columns], axis=1)

# Convert the 'occupation_status_unknown' to 'occupation_status_other'
X.loc[X['occupation_status_unknown'] == 1, 'occupation_status_other'] = 1

# Drop the 'occupation_status_unknown' column
X = X.drop(columns=['occupation_status_unknown'])

In [16]:
# Display the dataframe
X.info()
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age                                   204 non-null    int64  
 1   vhi_score                             204 non-null    int64  
 2   rsi_score                             204 non-null    int64  
 3   reflux_indicated                      204 non-null    int64  
 4   vhi_zscore                            204 non-null    float64
 5   vhi_impact                            204 non-null    int64  
 6   alcohol_consumption                   204 non-null    int64  
 7   alcohol_pd                            204 non-null    float64
 8   smoker                                204 non-null    int64  
 9   cigarettes_pd                         204 non-null    int64  
 10  carbonated_beverages                  204 non-null    int64  
 11  carbonated_pd      

,age,vhi_score,rsi_score,reflux_indicated,vhi_zscore,vhi_impact,alcohol_consumption,alcohol_pd,smoker,cigarettes_pd,...,gender_m,occupation_status_doctor,occupation_status_employee,occupation_status_housewife,occupation_status_other,occupation_status_pensioner,occupation_status_researcher,occupation_status_singer,occupation_status_student,occupation_status_technical operator
0,24,0,5,0,-0.58,0,1,0.36,0,0,...,1,0,0,0,1,0,0,0,0,0
1,60,80,10,0,4.76,4,0,0.00,0,0,...,1,0,0,0,1,0,0,0,0,0
2,22,0,10,0,-0.58,0,0,0.00,0,0,...,1,0,0,0,1,0,0,0,0,0
3,46,0,36,1,-0.58,0,1,0.36,2,15,...,0,0,0,1,0,0,0,0,0,0
4,51,19,15,1,0.68,1,1,0.36,0,0,...,0,0,0,0,0,0,1,0,0,0


In [50]:
# Collect info to output to JSON
output_dict = dict()

# Populate the dictionary
output_dict['feature_names'] = list(X.columns)
output_dict['smoker_map'] = smoker_map
output_dict['alcohol_map'] = alcohol_map
output_dict['habit_map'] = habit_map

# Convert the feature names to a JSON
json_data = json.dumps(output_dict)

# Specify the file path within your repository
file_path = '../voice_app/assets/model_meta.json'

# Export JSON data to a file
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

### Split and Scale

In [17]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [18]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# # Instantiate SMOTE
# smote = SMOTE()

# # Fit and transform the data
# X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

In [20]:
# Save the scaler
dump(X_scaler, '../voice_app/assets/scaler.joblib')

['../voice_app/assets/scaler.joblib']

## Hyperparameter Tuning

In [21]:
# Define the model parameters
number_input_features = len(X_train.columns)

# Maximum hidden layers (min. 2 for DL)
max_hidden_layers = 5

# Maximum neurons per hidden layer
max_num_neurons = number_input_features * 2 - 1

# Step count
step_count = 5

# Hidden layer activation functions
activation_functions = [
    'relu', 'leaky_relu', 'tanh',
    'elu', 'selu', 'exponential',
    'softmax', 'softplus'
]

# Define the output layer
output_layer_neurons = 1 # 4
output_layer_activation = 'sigmoid' # 'softmax'

# Model compilation
compile_loss = "binary_crossentropy"
# compile_loss = "sparse_categorical_crossentropy"
compile_opt = "adam"

# Number of max epochs
tuner_max_epochs = 20
search_max_epochs = 20

# Hyperband iterations
hp_iterations = 2

# Regularizers
reg_kernel = regularizers.L1(0.01)

In [22]:
# Extract class name and parameter value, for performance tracker
class_name = reg_kernel.__class__.__name__

if class_name == "L1L2":
    value_l1 = reg_kernel.get_config()['l1']
    value_l2 = reg_kernel.get_config()['l2']

    # Create string version
    reg_kernel_string = f"{class_name}(l1={value_l1:.3f})(l2={value_l2:.3f})"

else:
    param_value = reg_kernel.get_config()[f'{class_name.lower()}']

    # Create string version
    reg_kernel_string = f"{class_name}({param_value:.3f})"

print(reg_kernel_string)

L1(0.010)


In [23]:
# Initialise the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = tuner_max_epochs,
    hyperband_iterations = hp_iterations
)

In [24]:
# Find the best hyperparameters
tuner.search(
    X_train_scaled,
    y_train,
    epochs = search_max_epochs,
    validation_data = (X_test_scaled, y_test)
)
# tuner.search(
#     X_resampled,
#     y_resampled,
#     epochs = search_max_epochs,
#     validation_data = (X_test_scaled, y_test)
# )

Trial 60 Complete [00h 00m 01s]
val_accuracy: 0.7254902124404907

Best val_accuracy So Far: 0.8039215803146362
Total elapsed time: 00h 00m 33s


## Compile, Train, Evaluate the Best Model

In [25]:
# Get the top 3 model hyperparameters
top3_hyper = tuner.get_best_hyperparameters(3)

for model in top3_hyper:
    pprint(model.values)

{'activation_layer_0': 'softplus',
 'activation_layer_1': 'exponential',
 'activation_layer_2': 'elu',
 'activation_layer_3': 'elu',
 'activation_layer_4': 'tanh',
 'num_layers': 1,
 'tuner/bracket': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 1,
 'tuner/trial_id': '0049',
 'units_layer_0': 16,
 'units_layer_1': 36,
 'units_layer_2': 26,
 'units_layer_3': 36,
 'units_layer_4': 26}
{'activation_layer_0': 'tanh',
 'activation_layer_1': 'relu',
 'activation_layer_2': 'exponential',
 'activation_layer_3': 'softmax',
 'activation_layer_4': 'relu',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_layer_0': 31,
 'units_layer_1': 41,
 'units_layer_2': 46,
 'units_layer_3': 31,
 'units_layer_4': 31}
{'activation_layer_0': 'selu',
 'activation_layer_1': 'leaky_relu',
 'activation_layer_2': 'selu',
 'activation_layer_3': 'relu',
 'activation_layer_4': 'exponential',
 'num_layers': 1,
 'tuner/bracket': 0,
 'tune

In [26]:
# Get the top model
best_hyper = top3_hyper[0]
best_hyper.values

{'activation_layer_0': 'softplus',
 'units_layer_0': 16,
 'num_layers': 1,
 'units_layer_1': 36,
 'activation_layer_1': 'exponential',
 'units_layer_2': 26,
 'activation_layer_2': 'elu',
 'units_layer_3': 36,
 'activation_layer_3': 'elu',
 'units_layer_4': 26,
 'activation_layer_4': 'tanh',
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0049'}

In [27]:
# Define the model parameters
number_input_features = len(X_train.columns)
hidden_layer0_neurons = best_hyper.values['units_layer_0']
hidden_layer0_activation = best_hyper.values['activation_layer_0']

# Return the number of hidden layers
total_hidden = best_hyper.values['num_layers'] + 1

# Train model
model_train_epochs = 100

In [28]:
# Initialise the sequential model
nn = Sequential()

# Create the input layer and first hidden layer
nn.add(Dense(
    units = hidden_layer0_neurons,
    activation = hidden_layer0_activation,
    kernel_regularizer = reg_kernel,
    input_dim = number_input_features
))
        
# Create additional hidden layers
for layer in range(1, total_hidden):
    nn.add(Dense(
        units = best_hyper.values[f'units_layer_{layer}'],
        activation = best_hyper.values[f'activation_layer_{layer}'],
        kernel_regularizer = reg_kernel
    ))

# Create the output layer
nn.add(Dense(
    units = output_layer_neurons,
    activation = output_layer_activation
))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                544       
                                                                 
 dense_7 (Dense)             (None, 36)                612       
                                                                 
 dense_8 (Dense)             (None, 1)                 37        
                                                                 
Total params: 1193 (4.66 KB)
Trainable params: 1193 (4.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(
    loss = compile_loss,
    optimizer = compile_opt,
    metrics = ["accuracy"]
)

In [30]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    shuffle = True,
    epochs = model_train_epochs,
    verbose = 1
)
# fit_model = nn.fit(
#     X_resampled,
#     y_resampled,
#     shuffle = True,
#     epochs = model_train_epochs,
#     verbose = 1
# )

Epoch 1/100
5/5 [==============================] - 0s 999us/step - loss: 2.7185 - accuracy: 0.6993
Epoch 2/100
5/5 [==============================] - 0s 1ms/step - loss: 2.5769 - accuracy: 0.6797
Epoch 3/100
5/5 [==============================] - 0s 898us/step - loss: 2.4938 - accuracy: 0.6601
Epoch 4/100
5/5 [==============================] - 0s 945us/step - loss: 2.4327 - accuracy: 0.6601
Epoch 5/100
5/5 [==============================] - 0s 967us/step - loss: 2.3802 - accuracy: 0.6536
Epoch 6/100
5/5 [==============================] - 0s 923us/step - loss: 2.3299 - accuracy: 0.6667
Epoch 7/100
5/5 [==============================] - 0s 890us/step - loss: 2.2838 - accuracy: 0.6732
Epoch 8/100
5/5 [==============================] - 0s 921us/step - loss: 2.2442 - accuracy: 0.6928
Epoch 9/100
5/5 [==============================] - 0s 921us/step - loss: 2.2043 - accuracy: 0.6993
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 2.1688 - accuracy: 0.7124
Epoch 11/100


## Evaluate the Model Results

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 1.0219 - accuracy: 0.7255 - 43ms/epoch - 22ms/step
Loss: 1.0219465494155884, Accuracy: 0.7254902124404907


In [32]:
# Check the prediction's output probabilities
predicted_prob = nn.predict(X_test_scaled)
clean_prob = np.round(predicted_prob, 2)

# Round to the nearest integer and flatten
clean_predicted = np.round(predicted_prob).astype(int).flatten()

# Convert to a dataframe for readability
output_prob = pd.DataFrame({
    'Actual': y_test,
    'Predicted': clean_predicted,
    'Probability': clean_prob.flatten()
})

output_prob.head(10)

2/2 [==============================] - 0s 1ms/step


,Actual,Predicted,Probability
58,0,0,0.37
14,1,1,0.97
7,1,1,1.00
28,1,1,0.98
114,1,1,0.93
135,0,1,0.95
22,1,1,0.62
119,0,0,0.34
3,1,1,1.00
88,1,1,0.99


In [33]:
# Display the confusion matrix
cmatrix = confusion_matrix(y_test, clean_predicted)

# Convert the matrix to a DataFrame
cmatrix_df = pd.DataFrame(
    cmatrix,
    index = ["Actual 0", "Actual 1"],
    columns = ["Predicted 0", "Predicted 1"]
)

# Display the confusion matrix
print("Confusion Matrix:")
cmatrix_df

Confusion Matrix:


,Predicted 0,Predicted 1
Actual 0,6,8
Actual 1,6,31


In [34]:
# Print the classification report for the model
creport = classification_report(
    y_test,
    clean_predicted,
    target_names = ["Healthy (0)", "Pathological (1)"]
)

print("Classification Report:\n", creport)

Classification Report:
                   precision    recall  f1-score   support

     Healthy (0)       0.50      0.43      0.46        14
Pathological (1)       0.79      0.84      0.82        37

        accuracy                           0.73        51
       macro avg       0.65      0.63      0.64        51
    weighted avg       0.71      0.73      0.72        51



In [35]:
# Extract the precision and recall
precision = precision_score(y_test, clean_predicted, average=None)
recall = recall_score(y_test, clean_predicted, average=None)
print(precision, recall)

[0.5        0.79487179] [0.42857143 0.83783784]


## Save Results to Performance Tracker

In [36]:
# Create dictionary to save the results
results_dict = dict()

current_time = datetime.now()
results_dict['timestamp'] = current_time
results_dict['runtime'] = time.time() - start_time
results_dict['model_loss'] = model_loss
results_dict['model_accuracy'] = model_accuracy
results_dict['precision_0'] = precision[0]
results_dict['precision_1'] = precision[1]
results_dict['recall_0'] = recall[0]
results_dict['recall_1'] = recall[1]

In [37]:
# Print the model architecture
print(f"Input features: {number_input_features}")
print(f"Hidden Layer 0: {hidden_layer0_activation}, {hidden_layer0_neurons}")

# Populate results_dict with model details
results_dict['input_features'] = number_input_features
results_dict['num_layers'] = best_hyper.values['num_layers']
results_dict['hlayer_0_activation'] = hidden_layer0_activation
results_dict['hlayer_0_neurons'] = hidden_layer0_neurons
results_dict['kernel_regularizer'] = reg_kernel_string

for layer in range(1, total_hidden):
    activation = best_hyper.values[f'activation_layer_{layer}']
    neurons = best_hyper.values[f'units_layer_{layer}']
    print(f"Hidden Layer {layer}: {activation}, {neurons}")
    
    results_dict[f'hlayer_{layer}_activation'] = best_hyper.values[
        f'activation_layer_{layer}']
    results_dict[f'hlayer_{layer}_neurons'] = best_hyper.values[
        f'units_layer_{layer}']

results_dict['olayer_neurons'] = output_layer_neurons
results_dict['olayer_activation'] = output_layer_activation

# Populate results_dict with tuning details
results_dict['tuning_max_hidden'] = max_hidden_layers
results_dict['tuning_max_neurons'] = max_num_neurons
results_dict['tuning_step_count'] = step_count
results_dict['activation_functions'] = str(activation_functions)
results_dict['tuning_tuner_epochs'] = tuner_max_epochs
results_dict['tuning_search_epochs'] = search_max_epochs
results_dict['tuning_hp_iterations'] = hp_iterations

# Populate results_dict with compilation details
results_dict['compile_loss'] = compile_loss
results_dict['compile_optimizer'] = compile_opt

Input features: 33
Hidden Layer 0: softplus, 16
Hidden Layer 1: exponential, 36


In [38]:
# Change message
change_message = input("Changes from previous iteration: ")

# Append to results_dict
results_dict['change_message'] = change_message

Changes from previous iteration:  no change, run after notebook/database cleanup


In [39]:
# Display the dictionary
results_dict

{'timestamp': datetime.datetime(2024, 1, 7, 9, 27, 20, 380495),
 'runtime': 48.56920504570007,
 'model_loss': 1.0219465494155884,
 'model_accuracy': 0.7254902124404907,
 'precision_0': 0.5,
 'precision_1': 0.7948717948717948,
 'recall_0': 0.42857142857142855,
 'recall_1': 0.8378378378378378,
 'input_features': 33,
 'num_layers': 1,
 'hlayer_0_activation': 'softplus',
 'hlayer_0_neurons': 16,
 'kernel_regularizer': 'L1(0.010)',
 'hlayer_1_activation': 'exponential',
 'hlayer_1_neurons': 36,
 'olayer_neurons': 1,
 'olayer_activation': 'sigmoid',
 'tuning_max_hidden': 5,
 'tuning_max_neurons': 65,
 'tuning_step_count': 5,
 'activation_functions': "['relu', 'leaky_relu', 'tanh', 'elu', 'selu', 'exponential', 'softmax', 'softplus']",
 'tuning_tuner_epochs': 20,
 'tuning_search_epochs': 20,
 'tuning_hp_iterations': 2,
 'compile_loss': 'binary_crossentropy',
 'compile_optimizer': 'adam',
 'change_message': 'no change, run after notebook/database cleanup'}

In [40]:
# Convert the dictionary to a dataframe
results_df = pd.DataFrame(results_dict, index=[0])
results_df.head()

,timestamp,runtime,model_loss,model_accuracy,precision_0,precision_1,recall_0,recall_1,input_features,num_layers,...,tuning_max_hidden,tuning_max_neurons,tuning_step_count,activation_functions,tuning_tuner_epochs,tuning_search_epochs,tuning_hp_iterations,compile_loss,compile_optimizer,change_message
0,2024-01-07 09:27:20.380495,48.569205,1.021947,0.72549,0.5,0.794872,0.428571,0.837838,33,1,...,5,65,5,"['relu', 'leaky_relu', 'tanh', 'elu', 'selu', ...",20,20,2,binary_crossentropy,adam,"no change, run after notebook/database cleanup"


In [41]:
# Performance tracker
tracker_path = "../resources/tracker/dl_performance_tracker.csv"

# Model percentage
model_pct = round(model_accuracy, 3)

# Check if the CSV exists
if os.path.exists(tracker_path):
    
    # Read the existing CSV
    tracker_df = pd.read_csv(tracker_path)
    
    # Append the new row of data
    updated_df = pd.concat([tracker_df, results_df], ignore_index=True)
    
    # Update the CSV file
    updated_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/dl/run_{len(tracker_df)}_{model_pct}.h5', save_format='h5')

else:    
    # Export to CSV
    results_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/dl/run_0_{model_pct}.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
# Check the prediction's output probabilities
predicted_prob = nn.predict(X_test_scaled)
clean_prob = np.round(predicted_prob, 2)

# Round to the nearest integer and flatten
clean_predicted = np.round(predicted_prob).astype(int).flatten()

# Convert to a dataframe for readability
output_prob = pd.DataFrame({
    'Actual': y_test,
    'Predicted': clean_predicted,
    'Probability': clean_prob.flatten()
})
output_prob

2/2 [==============================] - 0s 967us/step


,Actual,Predicted,Probability
58,0,0,0.37
14,1,1,0.97
7,1,1,1.00
28,1,1,0.98
114,1,1,0.93
135,0,1,0.95
22,1,1,0.62
119,0,0,0.34
3,1,1,1.00
88,1,1,0.99


In [43]:
output_prob.loc[output_prob['Actual'] != output_prob['Predicted']]

,Actual,Predicted,Probability
135,0,1,0.95
179,1,0,0.19
166,0,1,0.94
163,1,0,0.13
46,0,1,0.55
67,1,0,0.47
134,0,1,0.73
83,0,1,0.65
142,0,1,1.00
113,1,0,0.23
